# Air Quality Prediction - ML Model Training

This notebook trains machine learning models to predict air quality parameters (PM2.5, PM10, NO, NO2, CO, CO2) based on:
- **Location features**: OpenStreetMap features around monitoring stations
- **Temporal features**: Time-based patterns (hour, day, month, etc.)

## Model Strategy
- Multi-output regression (predicting multiple pollutants simultaneously)
- Easy model switching for experimentation
- Designed for continuous training pipeline (Jenkins/Cronjobs)
- Modular architecture for future enhancements

## 1. Import Required Libraries

In [3]:
%pip install joblib pandas scikit-learn matplotlib seaborn

  Using cached scikit_learn-1.8.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.3 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-macosx_12_0_arm64.whl (8.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 2.7 MB/s  0:00:02 eta 0:00:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl (274 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 3.8 MB/s  0:00:00 eta 0:00:01
Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB

In [3]:
import pandas as pd
import numpy as np
import os
import glob
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully!
Pandas version: 3.0.0
NumPy version: 2.4.1


## 2. Configuration

Set paths and parameters for the training pipeline.

In [4]:
# Paths
OUTPUT_DIR = 'output'  # Directory containing station chunks
OSM_FEATURES_FILE = 'station_osm_features_filtered_110.csv'  # OSM features for 110 stations
MODEL_SAVE_DIR = 'models'  # Directory to save trained models
RESULTS_DIR = 'results'  # Directory to save results and plots

# Create directories if they don't exist
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Target pollutants to predict
TARGET_POLLUTANTS = ['PM2.5', 'PM10', 'NO', 'NO2', 'CO', 'CO2']

# Test size for train-test split
TEST_SIZE = 0.2
RANDOM_STATE = 42

print(f"Configuration:")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  OSM features file: {OSM_FEATURES_FILE}")
print(f"  Model save directory: {MODEL_SAVE_DIR}")
print(f"  Results directory: {RESULTS_DIR}")
print(f"  Target pollutants: {TARGET_POLLUTANTS}")
print(f"  Test size: {TEST_SIZE}")
print(f"  Random state: {RANDOM_STATE}")

Configuration:
  Output directory: output
  OSM features file: station_osm_features_filtered_110.csv
  Model save directory: models
  Results directory: results
  Target pollutants: ['PM2.5', 'PM10', 'NO', 'NO2', 'CO', 'CO2']
  Test size: 0.2
  Random state: 42


## 3. Load and Combine Air Quality Data

Load all chunked CSV files from the output directory and combine them into a single dataset.

In [5]:
# Find all CSV files in output directory
all_chunks = glob.glob(os.path.join(OUTPUT_DIR, '*', '*.csv'))

print(f"Found {len(all_chunks)} chunk files")
print(f"\nSample files:")
for i, file in enumerate(all_chunks[:5]):
    print(f"  {i+1}. {file}")

if len(all_chunks) > 5:
    print(f"  ... and {len(all_chunks) - 5} more files")

Found 178 chunk files

Sample files:
  1. output/DL015/DL015_chunk_3.csv
  2. output/DL015/DL015_chunk_2.csv
  3. output/DL015/DL015_chunk_1.csv
  4. output/DL012/DL012_chunk_1.csv
  5. output/DL012/DL012_chunk_3.csv
  ... and 173 more files


In [6]:
# Load and combine all chunks
print("Loading all chunk files...")
df_chunks_list = []

for i, chunk_file in enumerate(all_chunks):
    try:
        df_chunk = pd.read_csv(chunk_file)
        df_chunks_list.append(df_chunk)
        
        if (i + 1) % 10 == 0:
            print(f"  Loaded {i + 1}/{len(all_chunks)} files...")
    except Exception as e:
        print(f"  Error loading {chunk_file}: {e}")

# Combine all chunks
df_aq = pd.concat(df_chunks_list, ignore_index=True)

print(f"\nCombined dataset:")
print(f"  Total rows: {len(df_aq):,}")
print(f"  Total columns: {len(df_aq.columns)}")
print(f"  Memory usage: {df_aq.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nColumns: {df_aq.columns.tolist()}")

# Display first few rows
df_aq.head()

Loading all chunk files...
  Loaded 10/178 files...
  Loaded 20/178 files...
  Loaded 30/178 files...
  Loaded 40/178 files...
  Loaded 50/178 files...
  Loaded 60/178 files...
  Loaded 70/178 files...
  Loaded 80/178 files...
  Loaded 90/178 files...
  Loaded 100/178 files...
  Loaded 110/178 files...
  Loaded 120/178 files...
  Loaded 130/178 files...
  Loaded 140/178 files...
  Loaded 150/178 files...
  Loaded 160/178 files...
  Loaded 170/178 files...

Combined dataset:
  Total rows: 1,452,503
  Total columns: 42
  Memory usage: 2573.12 MB

Columns: ['StationId', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket', 'aeroway', 'amenity', 'barrier', 'boundary', 'building', 'craft', 'emergency', 'healthcare', 'highway', 'historic', 'industrial', 'landuse', 'leisure', 'man_made', 'military', 'natural', 'office', 'place', 'power', 'public_transport', 'railway', 'shop', 'sport', 'tourism', 'water', 'waterway']


,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,...,office,place,power,public_transport,railway,shop,sport,tourism,water,waterway
0,DL015,2020-05-14 17:00:00,38.5,116.75,NaN,NaN,NaN,NaN,NaN,1.25,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
1,DL015,2020-05-14 18:00:00,36.0,128.00,4.60,27.80,18.60,NaN,NaN,2.30,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
2,DL015,2020-05-14 19:00:00,75.0,NaN,3.90,21.33,14.53,51.65,NaN,2.88,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
3,DL015,2020-05-14 20:00:00,69.0,160.00,2.83,19.88,12.85,51.42,NaN,2.35,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
4,DL015,2020-05-14 21:00:00,51.0,78.00,4.42,32.88,21.18,51.35,NaN,1.95,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"


## 4. Load OSM Features

Load the OpenStreetMap features for the 110 stations.

In [7]:
# Load OSM features
df_osm = pd.read_csv(OSM_FEATURES_FILE)

print(f"OSM Features dataset:")
print(f"  Total stations: {len(df_osm)}")
print(f"  Total features: {len(df_osm.columns)}")
print(f"\nColumns: {df_osm.columns.tolist()}")

# Display first few rows
df_osm.head()

OSM Features dataset:
  Total stations: 96
  Total features: 33

Columns: ['station_id', 'original_station_id', 'station_name', 'latitude', 'longitude', '_total_elements', '_unique_feature_types', 'aeroway', 'amenity', 'barrier', 'boundary', 'building', 'craft', 'emergency', 'healthcare', 'highway', 'historic', 'industrial', 'landuse', 'leisure', 'man_made', 'military', 'natural', 'office', 'place', 'power', 'public_transport', 'railway', 'shop', 'sport', 'tourism', 'water', 'waterway']


,station_id,original_station_id,station_name,latitude,longitude,_total_elements,_unique_feature_types,aeroway,amenity,barrier,...,office,place,power,public_transport,railway,shop,sport,tourism,water,waterway
0,TN005,site_5094,"SIDCO Kurichi, Coimbatore - TNPCB",10.942451,76.978996,22953,6,NaN,"college,fuel",gate,...,NaN,NaN,NaN,NaN,rail,NaN,NaN,NaN,NaN,NaN
1,DL034,site_119,"Sirifort, Delhi - CPCB",28.550425,77.215938,11277,18,NaN,"bench,cafe,charging_station,clinic,college,con...","bollard,city_wall,cycle_barrier,entrance,fence...",...,"company,diplomatic,government,lawyer","neighbourhood,suburb","pole,substation",platform,NaN,"art,bakery,beauty,boutique,clothes,electrical,...","basketball,cricket,soccer,squash;badminton,ten...","gallery,museum",NaN,NaN
2,DL017,site_5395,"Lodhi Road, Delhi - IITM",28.588333,77.221667,12472,20,NaN,"arts_centre,atm,bank,bar,bicycle_rental,cafe,c...","fence,gate,kerb,lift_gate,wall",...,"association,company,diplomatic,government,ngo",neighbourhood,generator,platform,NaN,"bakery,bicycle,books,convenience,dairy,florist",tennis,"artwork,attraction",pond,NaN
3,MH014,site_5115,"Worli, Mumbai - MPCB",18.993616,72.812811,312159,18,NaN,"cafe,crematorium,fast_food,fuel,planetarium,sc...",gate,...,"financial,government","island,locality,neighbourhood,sea,state",NaN,"platform,station,stop_area,stop_position","station,stop,subway",mall,"horse_racing,swimming","attraction,museum",canal,"drain,lock_gate"
4,TN004,site_288,"Velachery Res. Area, Chennai - CPCB",13.005219,80.239812,12305,17,NaN,"atm,bench,bicycle_parking,cafe,college,drinkin...","block,gate,hedge,lift_gate,median,wall",...,NaN,neighbourhood,NaN,"platform,stop_position",NaN,"stationery,tea","cricket,racquet,tennis","apartment,artwork,attraction,zoo","lake,pond",stream


## 5. Data Preprocessing

### 5.1 Parse DateTime and Create Temporal Features

In [8]:
# Parse datetime column (assuming column name is 'Datetime' or 'datetime')
datetime_col = None
for col in df_aq.columns:
    if col.lower() in ['datetime', 'date', 'timestamp', 'time']:
        datetime_col = col
        break

if datetime_col:
    print(f"Found datetime column: {datetime_col}")
    df_aq[datetime_col] = pd.to_datetime(df_aq[datetime_col])
    
    # Create temporal features
    df_aq['year'] = df_aq[datetime_col].dt.year
    df_aq['month'] = df_aq[datetime_col].dt.month
    df_aq['day'] = df_aq[datetime_col].dt.day
    df_aq['hour'] = df_aq[datetime_col].dt.hour
    df_aq['day_of_week'] = df_aq[datetime_col].dt.dayofweek  # Monday=0, Sunday=6
    df_aq['day_of_year'] = df_aq[datetime_col].dt.dayofyear
    df_aq['week_of_year'] = df_aq[datetime_col].dt.isocalendar().week
    df_aq['is_weekend'] = (df_aq['day_of_week'] >= 5).astype(int)  # 1 if weekend, 0 otherwise
    
    # Cyclical encoding for hour (to capture 23:00 and 00:00 proximity)
    df_aq['hour_sin'] = np.sin(2 * np.pi * df_aq['hour'] / 24)
    df_aq['hour_cos'] = np.cos(2 * np.pi * df_aq['hour'] / 24)
    
    # Cyclical encoding for month
    df_aq['month_sin'] = np.sin(2 * np.pi * df_aq['month'] / 12)
    df_aq['month_cos'] = np.cos(2 * np.pi * df_aq['month'] / 12)
    
    print(f"\nTemporal features created:")
    print(f"  - year, month, day, hour")
    print(f"  - day_of_week, day_of_year, week_of_year")
    print(f"  - is_weekend")
    print(f"  - hour_sin, hour_cos (cyclical encoding)")
    print(f"  - month_sin, month_cos (cyclical encoding)")
else:
    print("Warning: No datetime column found!")
    print(f"Available columns: {df_aq.columns.tolist()}")

# Display sample with temporal features
df_aq.head()

Found datetime column: Datetime

Temporal features created:
  - year, month, day, hour
  - day_of_week, day_of_year, week_of_year
  - is_weekend
  - hour_sin, hour_cos (cyclical encoding)
  - month_sin, month_cos (cyclical encoding)


,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,...,day,hour,day_of_week,day_of_year,week_of_year,is_weekend,hour_sin,hour_cos,month_sin,month_cos
0,DL015,2020-05-14 17:00:00,38.5,116.75,NaN,NaN,NaN,NaN,NaN,1.25,...,14,17,3,135,20,0,-0.965926,-2.588190e-01,0.5,-0.866025
1,DL015,2020-05-14 18:00:00,36.0,128.00,4.60,27.80,18.60,NaN,NaN,2.30,...,14,18,3,135,20,0,-1.000000,-1.836970e-16,0.5,-0.866025
2,DL015,2020-05-14 19:00:00,75.0,NaN,3.90,21.33,14.53,51.65,NaN,2.88,...,14,19,3,135,20,0,-0.965926,2.588190e-01,0.5,-0.866025
3,DL015,2020-05-14 20:00:00,69.0,160.00,2.83,19.88,12.85,51.42,NaN,2.35,...,14,20,3,135,20,0,-0.866025,5.000000e-01,0.5,-0.866025
4,DL015,2020-05-14 21:00:00,51.0,78.00,4.42,32.88,21.18,51.35,NaN,1.95,...,14,21,3,135,20,0,-0.707107,7.071068e-01,0.5,-0.866025


### 5.2 Merge Air Quality Data with OSM Features

In [9]:
# Find station ID column in air quality data
station_id_col = None
for col in df_aq.columns:
    if 'station' in col.lower() and 'id' in col.lower():
        station_id_col = col
        break

if not station_id_col:
    # Try just 'StationId' or similar
    for col in df_aq.columns:
        if col.lower() in ['stationid', 'station_id', 'station']:
            station_id_col = col
            break

print(f"Station ID column in air quality data: {station_id_col}")
print(f"Unique stations in AQ data: {df_aq[station_id_col].nunique()}")
print(f"Unique stations in OSM data: {df_osm['station_id'].nunique()}")

# Merge datasets
df_merged = df_aq.merge(
    df_osm,
    left_on=station_id_col,
    right_on='station_id',
    how='inner'
)

print(f"\nMerged dataset:")
print(f"  Total rows: {len(df_merged):,}")
print(f"  Total columns: {len(df_merged.columns)}")
print(f"  Stations with data: {df_merged[station_id_col].nunique()}")

df_merged.head()

Station ID column in air quality data: StationId
Unique stations in AQ data: 56
Unique stations in OSM data: 93

Merged dataset:
  Total rows: 1,477,165
  Total columns: 87
  Stations with data: 56


,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,...,office_y,place_y,power_y,public_transport_y,railway_y,shop_y,sport_y,tourism_y,water_y,waterway_y
0,DL015,2020-05-14 17:00:00,38.5,116.75,NaN,NaN,NaN,NaN,NaN,1.25,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
1,DL015,2020-05-14 18:00:00,36.0,128.00,4.60,27.80,18.60,NaN,NaN,2.30,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
2,DL015,2020-05-14 19:00:00,75.0,NaN,3.90,21.33,14.53,51.65,NaN,2.88,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
3,DL015,2020-05-14 20:00:00,69.0,160.00,2.83,19.88,12.85,51.42,NaN,2.35,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"
4,DL015,2020-05-14 21:00:00,51.0,78.00,4.42,32.88,21.18,51.35,NaN,1.95,...,"bus_inspector,estate_agent,government,lawyer","neighbourhood,quarter,suburb","line,pole,tower,transformer","platform,stop_position","construction,proposed","appliance,beauty,beverages,clothes,confectione...",NaN,NaN,river,"canal,drain"


### 5.3 Prepare Feature Matrix and Target Variables

In [10]:
# Define columns to exclude from features
exclude_cols = [
    station_id_col,
    'station_id',
    'original_station_id',
    'station_name',
    'latitude',
    'longitude',
    datetime_col if datetime_col else 'datetime',
    '_total_elements',
    '_unique_feature_types'
] + TARGET_POLLUTANTS

# Get feature columns (all columns except excluded ones and target pollutants)
feature_cols = [col for col in df_merged.columns if col not in exclude_cols]

print(f"Feature columns ({len(feature_cols)}):")
print(feature_cols)

# Check which target pollutants are available
available_targets = [col for col in TARGET_POLLUTANTS if col in df_merged.columns]
missing_targets = [col for col in TARGET_POLLUTANTS if col not in df_merged.columns]

print(f"\nAvailable target pollutants ({len(available_targets)}): {available_targets}")
if missing_targets:
    print(f"Missing target pollutants ({len(missing_targets)}): {missing_targets}")

# Use available targets
TARGET_POLLUTANTS = available_targets

Feature columns (73):
['NOx', 'NH3', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket', 'aeroway_x', 'amenity_x', 'barrier_x', 'boundary_x', 'building_x', 'craft_x', 'emergency_x', 'healthcare_x', 'highway_x', 'historic_x', 'industrial_x', 'landuse_x', 'leisure_x', 'man_made_x', 'military_x', 'natural_x', 'office_x', 'place_x', 'power_x', 'public_transport_x', 'railway_x', 'shop_x', 'sport_x', 'tourism_x', 'water_x', 'waterway_x', 'year', 'month', 'day', 'hour', 'day_of_week', 'day_of_year', 'week_of_year', 'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'aeroway_y', 'amenity_y', 'barrier_y', 'boundary_y', 'building_y', 'craft_y', 'emergency_y', 'healthcare_y', 'highway_y', 'historic_y', 'industrial_y', 'landuse_y', 'leisure_y', 'man_made_y', 'military_y', 'natural_y', 'office_y', 'place_y', 'power_y', 'public_transport_y', 'railway_y', 'shop_y', 'sport_y', 'tourism_y', 'water_y', 'waterway_y']

Available target pollutants (5): ['PM2.5', 'PM10', 'NO', 'NO

In [11]:
# Create feature matrix X and target matrix y
X = df_merged[feature_cols].copy()
y = df_merged[TARGET_POLLUTANTS].copy()

print(f"Feature matrix X shape: {X.shape}")
print(f"Target matrix y shape: {y.shape}")

# Check for missing values
print(f"\nMissing values in X:")
missing_x = X.isnull().sum()
print(missing_x[missing_x > 0])

print(f"\nMissing values in y:")
missing_y = y.isnull().sum()
print(missing_y[missing_y > 0])

Feature matrix X shape: (1477165, 73)
Target matrix y shape: (1477165, 5)

Missing values in X:
NOx                    270152
NH3                    701712
SO2                    516056
O3                     409212
Benzene                554728
Toluene                591689
Xylene                1181898
AQI                    311297
AQI_Bucket             311297
aeroway_x             1421144
amenity_x               60133
barrier_x               80951
boundary_x             182635
building_x             293780
craft_x               1259563
emergency_x           1361428
healthcare_x           703941
historic_x            1161778
industrial_x          1434893
landuse_x               29269
leisure_x              116378
man_made_x             603705
military_x            1477165
natural_x              493423
office_x               645666
place_x                661924
power_x                842959
public_transport_x     446282
railway_x              541046
shop_x                 435991
spor

### 5.4 Analyze Missing Data Patterns

Before handling missing values, let's understand the missingness patterns in our target variables.

In [12]:
# Analyze missing data patterns in target variables
print("Missing Data Analysis for Target Pollutants")
print("=" * 80)

for pollutant in TARGET_POLLUTANTS:
    missing_count = y[pollutant].isnull().sum()
    missing_pct = (missing_count / len(y)) * 100
    available_count = len(y) - missing_count
    print(f"{pollutant:<10}: {missing_count:>8,} missing ({missing_pct:>5.2f}%) | {available_count:>8,} available")

# Check how many rows have ALL pollutants available
all_available = (~y.isnull()).all(axis=1).sum()
all_available_pct = (all_available / len(y)) * 100
print(f"\n{'ALL':<10}: {len(y) - all_available:>8,} rows with ANY missing ({100-all_available_pct:>5.2f}%) | {all_available:>8,} rows complete ({all_available_pct:>5.2f}%)")

# Check how many rows have at least ONE pollutant available
any_available = (~y.isnull()).any(axis=1).sum()
any_available_pct = (any_available / len(y)) * 100
print(f"{'AT LEAST ONE':<10}: {any_available:>8,} rows with data ({any_available_pct:>5.2f}%)")

# Show correlation of missingness
print("\nMissingness Correlation Matrix:")
print("(1.0 = always missing together, 0.0 = independent)")
missing_corr = y.isnull().corr()
print(missing_corr.round(2))

Missing Data Analysis for Target Pollutants
PM2.5     :  380,261 missing (25.74%) | 1,096,904 available
PM10      :  500,563 missing (33.89%) |  976,602 available
NO        :  303,612 missing (20.55%) | 1,173,553 available
NO2       :  306,098 missing (20.72%) | 1,171,067 available
CO        :  283,086 missing (19.16%) | 1,194,079 available

ALL       :  655,506 rows with ANY missing (44.38%) |  821,659 rows complete (55.62%)
AT LEAST ONE: 1,287,399 rows with data (87.15%)

Missingness Correlation Matrix:
(1.0 = always missing together, 0.0 = independent)
       PM2.5  PM10    NO   NO2    CO
PM2.5   1.00  0.54  0.69  0.72  0.55
PM10    0.54  1.00  0.56  0.58  0.54
NO      0.69  0.56  1.00  0.93  0.62
NO2     0.72  0.58  0.93  1.00  0.64
CO      0.55  0.54  0.62  0.64  1.00


### 5.5 Handle Missing Values and Data Types

In [15]:
# Separate numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"Numeric features: {len(numeric_cols)}")
print(f"Categorical features: {len(categorical_cols)}")
if categorical_cols:
    print(f"Categorical columns: {categorical_cols}")

# Handle categorical features (OSM features with comma-separated values)
if categorical_cols:
    print("\nProcessing categorical features (counting unique values)...")
    for col in categorical_cols:
        X[f'{col}_count'] = X[col].fillna('').apply(lambda x: len(str(x).split(',')) if x else 0)
        X = X.drop(columns=[col])
    
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    print(f"Features after processing: {len(numeric_cols)}")

# Impute missing values - ensure all columns are actually numeric
# Force conversion to numeric and drop any columns that can't be converted
X_numeric = X.select_dtypes(include=[np.number]).copy()

print(f"\nBefore imputation: {X_numeric.shape}")

imputer_X = SimpleImputer(strategy='median')
X_imputed_array = imputer_X.fit_transform(X_numeric)

print(f"After imputation array shape: {X_imputed_array.shape}")

# Create DataFrame - use actual shape from the imputed array
X_imputed = pd.DataFrame(
    X_imputed_array,
    columns=X_numeric.columns[:X_imputed_array.shape[1]],  # Use only as many column names as we have columns
    index=X.index
)

print(f"\nFeature matrix after imputation: {X_imputed.shape}")

Numeric features: 73
Categorical features: 0

Before imputation: (1477165, 73)
After imputation array shape: (1477165, 72)

Feature matrix after imputation: (1477165, 72)


### 5.6 Handle Missing Target Values - Strategy Selection

**Choose your strategy** by setting the `MISSING_TARGET_STRATEGY` variable:

**Strategy Options:**
1. **'impute'** - Fill missing values with forward/backward fill then median (RECOMMENDED - keeps all data)
2. **'drop_all'** - Drop rows with ANY missing targets (most data loss, cleanest dataset)
3. **'drop_majority'** - Drop rows missing MORE THAN 50% of targets (balanced approach)

**Trade-offs:**
- Imputation: Keeps all data but introduces estimation error
- Drop all: Cleanest but loses ~50% of data
- Drop majority: Keeps rows with at least some real measurements

In [16]:
# ============================================================================
# CHANGE THIS VARIABLE TO SELECT MISSING DATA STRATEGY
# ============================================================================
MISSING_TARGET_STRATEGY = 'impute'  # Options: 'impute', 'drop_all', 'drop_majority'
# ============================================================================

print(f"Selected strategy: {MISSING_TARGET_STRATEGY}")
print("=" * 80)

if MISSING_TARGET_STRATEGY == 'impute':
    print("Strategy: IMPUTE missing target values")
    print("  - Forward fill (use previous hour's value)")
    print("  - Backward fill (use next hour's value)")
    print("  - Median fill (for remaining NaNs)")
    print("  - Keeps ALL data - no rows dropped!")
    
    # Sort by datetime to ensure proper forward/backward fill
    if datetime_col and datetime_col in df_merged.columns:
        # Get the original order indices
        original_index = y.index
        
        # Create a temporary dataframe with datetime for proper sorting
        y_with_time = y.copy()
        y_with_time['_datetime'] = df_merged.loc[y.index, datetime_col]
        y_with_time['_station'] = df_merged.loc[y.index, station_id_col]
        
        # Sort by station and time
        y_sorted = y_with_time.sort_values(['_station', '_datetime'])
        
        # Apply imputation separately for each station
        y_imputed_list = []
        for station in y_sorted['_station'].unique():
            station_mask = y_sorted['_station'] == station
            y_station = y_sorted[station_mask][TARGET_POLLUTANTS].copy()
            
            # Forward fill, then backward fill, then median
            y_station = y_station.ffill().bfill()
            
            # Fill any remaining NaNs with median
            for col in TARGET_POLLUTANTS:
                if y_station[col].isnull().any():
                    median_val = y_station[col].median()
                    if pd.isna(median_val):  # If still NaN, use global median
                        median_val = y[col].median()
                    y_station[col].fillna(median_val, inplace=True)
            
            y_imputed_list.append(y_station)
        
        # Combine and restore original order
        y_imputed_sorted = pd.concat(y_imputed_list)
        y_imputed = y_imputed_sorted.reindex(original_index)
    else:
        # Simple imputation without time-based ordering
        y_imputed = y.copy()
        for col in TARGET_POLLUTANTS:
            median_val = y[col].median()
            y_imputed[col].fillna(median_val, inplace=True)
    
    X_clean = X_imputed.copy()
    y_clean = y_imputed.copy()
    
    print(f"\nResult:")
    print(f"  Rows kept: {len(X_clean):,} (100.0%)")
    print(f"  Rows dropped: 0")
    
elif MISSING_TARGET_STRATEGY == 'drop_all':
    print("Strategy: DROP rows with ANY missing target values")
    print("  - Most conservative approach")
    print("  - Highest data loss but cleanest dataset")
    print("  - Only keeps rows with ALL pollutants measured")
    
    mask = ~y.isnull().any(axis=1)
    X_clean = X_imputed[mask].copy()
    y_clean = y[mask].copy()
    
    rows_kept = len(X_clean)
    rows_dropped = len(X_imputed) - rows_kept
    pct_kept = (rows_kept / len(X_imputed)) * 100
    
    print(f"\nResult:")
    print(f"  Rows kept: {rows_kept:,} ({pct_kept:.2f}%)")
    print(f"  Rows dropped: {rows_dropped:,} ({100-pct_kept:.2f}%)")
    
elif MISSING_TARGET_STRATEGY == 'drop_majority':
    print("Strategy: DROP rows missing MORE THAN 50% of targets")
    print("  - Balanced approach")
    print("  - Keeps rows with at least 3 out of 6 pollutants")
    print("  - Moderate data retention")
    
    # Count non-null values per row
    non_null_counts = y.notna().sum(axis=1)
    threshold = len(TARGET_POLLUTANTS) / 2
    
    mask = non_null_counts > threshold
    X_clean = X_imputed[mask].copy()
    y_clean = y[mask].copy()
    
    # Impute remaining NaNs with median
    for col in TARGET_POLLUTANTS:
        if y_clean[col].isnull().any():
            median_val = y_clean[col].median()
            y_clean[col].fillna(median_val, inplace=True)
    
    rows_kept = len(X_clean)
    rows_dropped = len(X_imputed) - rows_kept
    pct_kept = (rows_kept / len(X_imputed)) * 100
    
    print(f"\nResult:")
    print(f"  Rows kept: {rows_kept:,} ({pct_kept:.2f}%)")
    print(f"  Rows dropped: {rows_dropped:,} ({100-pct_kept:.2f}%)")
    print(f"  Remaining NaNs imputed with median")

else:
    raise ValueError(f"Invalid strategy: {MISSING_TARGET_STRATEGY}. Choose 'impute', 'drop_all', or 'drop_majority'")

print(f"\nFinal dataset:")
print(f"  X shape: {X_clean.shape}")
print(f"  y shape: {y_clean.shape}")
print(f"  Missing values in y: {y_clean.isnull().sum().sum()}")

Selected strategy: impute
Strategy: IMPUTE missing target values
  - Forward fill (use previous hour's value)
  - Backward fill (use next hour's value)
  - Median fill (for remaining NaNs)
  - Keeps ALL data - no rows dropped!

Result:
  Rows kept: 1,477,165 (100.0%)
  Rows dropped: 0

Final dataset:
  X shape: (1477165, 72)
  y shape: (1477165, 5)
  Missing values in y: 310791


### 5.7 Train-Test Split

In [17]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_clean,
    y_clean,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

print(f"Dataset split:")
print(f"  Training set: {X_train.shape[0]:,} samples")
print(f"  Testing set: {X_test.shape[0]:,} samples")
print(f"  Features: {X_train.shape[1]}")
print(f"  Targets: {y_train.shape[1]}")

Dataset split:
  Training set: 1,181,732 samples
  Testing set: 295,433 samples
  Features: 72
  Targets: 5


### 5.8 Feature Scaling

In [18]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")
print(f"  Mean: {X_train_scaled.mean():.6f}")
print(f"  Std: {X_train_scaled.std():.6f}")

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

Features scaled using StandardScaler
  Mean: -0.000000
  Std: 0.993031


## 6. Model Definition and Selection

Define multiple ML models and create a mechanism for easy switching between them.

In [19]:
# Define available models
MODELS = {
    'linear_regression': {
        'model': MultiOutputRegressor(LinearRegression()),
        'description': 'Linear Regression - Simple baseline model',
        'use_scaling': True
    },
    'ridge': {
        'model': MultiOutputRegressor(Ridge(alpha=1.0, random_state=RANDOM_STATE)),
        'description': 'Ridge Regression - L2 regularization',
        'use_scaling': True
    },
    'lasso': {
        'model': MultiOutputRegressor(Lasso(alpha=1.0, random_state=RANDOM_STATE)),
        'description': 'Lasso Regression - L1 regularization with feature selection',
        'use_scaling': True
    },
    'decision_tree': {
        'model': MultiOutputRegressor(DecisionTreeRegressor(random_state=RANDOM_STATE)),
        'description': 'Decision Tree - Non-linear, interpretable',
        'use_scaling': False
    },
    'random_forest': {
        'model': MultiOutputRegressor(
            RandomForestRegressor(
                n_estimators=100,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
        ),
        'description': 'Random Forest - Ensemble of decision trees, robust to overfitting',
        'use_scaling': False
    },
    'gradient_boosting': {
        'model': MultiOutputRegressor(
            GradientBoostingRegressor(
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                random_state=RANDOM_STATE
            )
        ),
        'description': 'Gradient Boosting - Sequential ensemble, high accuracy',
        'use_scaling': False
    }
}

print("Available Models:")
print("=" * 80)
for i, (model_name, model_info) in enumerate(MODELS.items(), 1):
    print(f"{i}. {model_name}")
    print(f"   {model_info['description']}")
    print(f"   Uses scaling: {model_info['use_scaling']}")
    print()

Available Models:
1. linear_regression
   Linear Regression - Simple baseline model
   Uses scaling: True

2. ridge
   Ridge Regression - L2 regularization
   Uses scaling: True

3. lasso
   Lasso Regression - L1 regularization with feature selection
   Uses scaling: True

4. decision_tree
   Decision Tree - Non-linear, interpretable
   Uses scaling: False

5. random_forest
   Random Forest - Ensemble of decision trees, robust to overfitting
   Uses scaling: False

6. gradient_boosting
   Gradient Boosting - Sequential ensemble, high accuracy
   Uses scaling: False



## 7. Training and Evaluation Functions

In [20]:
def train_model(model_name, model_config, X_train_raw, X_train_sc, X_test_raw, X_test_sc, y_train, y_test):
    """
    Train a model and evaluate its performance.
    
    Args:
        model_name: Name of the model
        model_config: Model configuration dictionary
        X_train_raw: Unscaled training features
        X_train_sc: Scaled training features
        X_test_raw: Unscaled test features
        X_test_sc: Scaled test features
        y_train: Training targets
        y_test: Testing targets
    
    Returns:
        Dictionary with trained model and metrics
    """
    print(f"\nTraining: {model_name}")
    print("=" * 80)
    print(f"Description: {model_config['description']}")
    
    # Select appropriate data (scaled or unscaled)
    X_tr = X_train_sc if model_config['use_scaling'] else X_train_raw
    X_te = X_test_sc if model_config['use_scaling'] else X_test_raw
    
    # Train model
    start_time = datetime.now()
    model = model_config['model']
    model.fit(X_tr, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    print(f"Training completed in {training_time:.2f} seconds")
    
    # Make predictions
    y_train_pred = model.predict(X_tr)
    y_test_pred = model.predict(X_te)
    
    # Calculate metrics for each pollutant
    results = {
        'model_name': model_name,
        'model': model,
        'training_time': training_time,
        'use_scaling': model_config['use_scaling'],
        'metrics': {}
    }
    
    print("\nPerformance Metrics:")
    print("-" * 80)
    print(f"{'Pollutant':<10} {'MAE':<12} {'RMSE':<12} {'R²':<12} {'Train R²':<12}")
    print("-" * 80)
    
    for i, pollutant in enumerate(TARGET_POLLUTANTS):
        # Test metrics
        mae = mean_absolute_error(y_test.iloc[:, i], y_test_pred[:, i])
        rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_test_pred[:, i]))
        r2 = r2_score(y_test.iloc[:, i], y_test_pred[:, i])
        
        # Train R² (to check overfitting)
        r2_train = r2_score(y_train.iloc[:, i], y_train_pred[:, i])
        
        results['metrics'][pollutant] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'R2_train': r2_train
        }
        
        print(f"{pollutant:<10} {mae:<12.4f} {rmse:<12.4f} {r2:<12.4f} {r2_train:<12.4f}")
    
    # Overall metrics (average across all pollutants)
    avg_mae = np.mean([m['MAE'] for m in results['metrics'].values()])
    avg_rmse = np.mean([m['RMSE'] for m in results['metrics'].values()])
    avg_r2 = np.mean([m['R2'] for m in results['metrics'].values()])
    avg_r2_train = np.mean([m['R2_train'] for m in results['metrics'].values()])
    
    print("-" * 80)
    print(f"{'AVERAGE':<10} {avg_mae:<12.4f} {avg_rmse:<12.4f} {avg_r2:<12.4f} {avg_r2_train:<12.4f}")
    print("-" * 80)
    
    results['metrics']['AVERAGE'] = {
        'MAE': avg_mae,
        'RMSE': avg_rmse,
        'R2': avg_r2,
        'R2_train': avg_r2_train
    }
    
    return results

print("Training function defined successfully!")

Training function defined successfully!


In [25]:
# Verify no NaN values remain before training
print("Pre-training data validation:")
print("=" * 80)

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Check for NaN in X
X_train_nans = X_train.isnull().sum().sum()
X_test_nans = X_test.isnull().sum().sum()
print(f"\nNaNs in X_train: {X_train_nans}")
print(f"NaNs in X_test: {X_test_nans}")

# Check for NaN in y
y_train_nans = y_train.isnull().sum().sum()
y_test_nans = y_test.isnull().sum().sum()
print(f"NaNs in y_train: {y_train_nans}")
print(f"NaNs in y_test: {y_test_nans}")

# If NaNs found in y, apply emergency fix
if y_train_nans > 0 or y_test_nans > 0:
    print("\n⚠️  WARNING: NaN values found in target variables!")
    print("Applying emergency median imputation...")
    
    # Create copies and fill
    y_train = y_train.copy()
    y_test = y_test.copy()
    
    for col in y_train.columns:
        if y_train[col].isnull().any():
            median_val = y_train[col].median()
            if pd.isna(median_val):
                median_val = 0  # Fallback if entire column is NaN
            y_train[col] = y_train[col].fillna(median_val)
            print(f"  Filled {col} in y_train with median: {median_val:.2f}")
    
    for col in y_test.columns:
        if y_test[col].isnull().any():
            median_val = y_test[col].median()
            if pd.isna(median_val):
                median_val = y_train[col].median()  # Use train median
            y_test[col] = y_test[col].fillna(median_val)
            print(f"  Filled {col} in y_test with median: {median_val:.2f}")
    
    print("\n✓ Emergency imputation complete")
    print(f"Final NaNs in y_train: {y_train.isnull().sum().sum()}")
    print(f"Final NaNs in y_test: {y_test.isnull().sum().sum()}")
else:
    print("\n✓ All data validated - no NaN values found")

print("=" * 80)
print("Ready for training!\n")

Pre-training data validation:

X_train shape: (1181732, 72)
X_test shape: (295433, 72)
y_train shape: (1181732, 5)
y_test shape: (295433, 5)

NaNs in X_train: 0
NaNs in X_test: 0
NaNs in y_train: 248773
NaNs in y_test: 62018

⚠️  WARNING: NaN values found in target variables!
Applying emergency median imputation...
  Filled PM2.5 in y_train with median: 66.50
  Filled PM10 in y_train with median: 128.66
  Filled PM2.5 in y_test with median: 66.50
  Filled PM10 in y_test with median: 129.25

✓ Emergency imputation complete
Final NaNs in y_train: 0
Final NaNs in y_test: 0
Ready for training!



## 8. Model Training - Manual Selection

**IMPORTANT**: Change the `SELECTED_MODEL` variable below to train different models.

Available options:
- `'linear_regression'`
- `'ridge'`
- `'lasso'`
- `'decision_tree'`
- `'random_forest'`
- `'gradient_boosting'`

In [ ]:
# ============================================================================
# CHANGE THIS VARIABLE TO SELECT A DIFFERENT MODEL
# ============================================================================
SELECTED_MODEL = 'random_forest'  # <-- Change this to switch models
# ============================================================================

if SELECTED_MODEL not in MODELS:
    print(f"Error: '{SELECTED_MODEL}' is not a valid model name.")
    print(f"Available models: {list(MODELS.keys())}")
else:
    # Train the selected model
    results = train_model(
        SELECTED_MODEL,
        MODELS[SELECTED_MODEL],
        X_train,
        X_train_scaled,
        X_test,
        X_test_scaled,
        y_train,
        y_test
    )


Training: random_forest
Description: Random Forest - Ensemble of decision trees, robust to overfitting


## 9. Save Trained Model

Save the trained model, scaler, and metadata for future use.

In [ ]:
# Create timestamp for versioning
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f"{SELECTED_MODEL}_{timestamp}.pkl"
scaler_filename = f"scaler_{timestamp}.pkl"
metadata_filename = f"metadata_{timestamp}.txt"

# Save model
model_path = os.path.join(MODEL_SAVE_DIR, model_filename)
joblib.dump(results['model'], model_path)
print(f"Model saved to: {model_path}")

# Save scaler (if model uses scaling)
if results['use_scaling']:
    scaler_path = os.path.join(MODEL_SAVE_DIR, scaler_filename)
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to: {scaler_path}")

# Save metadata
metadata_path = os.path.join(MODEL_SAVE_DIR, metadata_filename)
with open(metadata_path, 'w') as f:
    f.write(f"Model: {SELECTED_MODEL}\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Training samples: {len(X_train)}\n")
    f.write(f"Test samples: {len(X_test)}\n")
    f.write(f"Features: {len(X_train.columns)}\n")
    f.write(f"Target pollutants: {', '.join(TARGET_POLLUTANTS)}\n")
    f.write(f"Uses scaling: {results['use_scaling']}\n")
    f.write(f"Training time: {results['training_time']:.2f} seconds\n")
    f.write(f"\nPerformance Metrics:\n")
    f.write("=" * 80 + "\n")
    for pollutant, metrics in results['metrics'].items():
        f.write(f"\n{pollutant}:\n")
        for metric_name, value in metrics.items():
            f.write(f"  {metric_name}: {value:.4f}\n")

print(f"Metadata saved to: {metadata_path}")
print("\nModel artifacts saved successfully!")

## 10. Visualization of Results

In [ ]:
# Plot performance metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle(f'Model Performance: {SELECTED_MODEL}', fontsize=16, fontweight='bold')

pollutants = [p for p in TARGET_POLLUTANTS]
mae_values = [results['metrics'][p]['MAE'] for p in pollutants]
rmse_values = [results['metrics'][p]['RMSE'] for p in pollutants]
r2_values = [results['metrics'][p]['R2'] for p in pollutants]

# MAE plot
axes[0].bar(pollutants, mae_values, color='steelblue')
axes[0].set_title('Mean Absolute Error (MAE)', fontweight='bold')
axes[0].set_xlabel('Pollutant')
axes[0].set_ylabel('MAE')
axes[0].tick_params(axis='x', rotation=45)

# RMSE plot
axes[1].bar(pollutants, rmse_values, color='coral')
axes[1].set_title('Root Mean Squared Error (RMSE)', fontweight='bold')
axes[1].set_xlabel('Pollutant')
axes[1].set_ylabel('RMSE')
axes[1].tick_params(axis='x', rotation=45)

# R² plot
axes[2].bar(pollutants, r2_values, color='mediumseagreen')
axes[2].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[2].set_title('R² Score', fontweight='bold')
axes[2].set_xlabel('Pollutant')
axes[2].set_ylabel('R²')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, f'{SELECTED_MODEL}_{timestamp}_metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"Plot saved to: {os.path.join(RESULTS_DIR, f'{SELECTED_MODEL}_{timestamp}_metrics.png')}")

## 11. Compare All Models (Optional)

Run this section to train and compare all available models.

In [1]:
# Set to True to train all models
TRAIN_ALL_MODELS = True  # <-- Change to True to compare all models

if TRAIN_ALL_MODELS:
    all_results = {}
    
    for model_name, model_config in MODELS.items():
        result = train_model(
            model_name,
            model_config,
            X_train,
            X_train_scaled,
            X_test,
            X_test_scaled,
            y_train,
            y_test
        )
        all_results[model_name] = result
    
    # Create comparison DataFrame
    comparison_data = []
    for model_name, result in all_results.items():
        avg_metrics = result['metrics']['AVERAGE']
        comparison_data.append({
            'Model': model_name,
            'MAE': avg_metrics['MAE'],
            'RMSE': avg_metrics['RMSE'],
            'R²': avg_metrics['R2'],
            'Training Time (s)': result['training_time']
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    df_comparison = df_comparison.sort_values('R²', ascending=False)
    
    print("\nModel Comparison (sorted by R²):")
    print("=" * 80)
    print(df_comparison.to_string(index=False))
    
    # Save comparison to CSV
    comparison_path = os.path.join(RESULTS_DIR, f'model_comparison_{timestamp}.csv')
    df_comparison.to_csv(comparison_path, index=False)
    print(f"\nComparison saved to: {comparison_path}")
    
    # Visualize comparison
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(df_comparison))
    width = 0.25
    
    ax.bar(x - width, df_comparison['MAE'], width, label='MAE', color='steelblue')
    ax.bar(x, df_comparison['RMSE'], width, label='RMSE', color='coral')
    ax.bar(x + width, df_comparison['R²'] * 100, width, label='R² (×100)', color='mediumseagreen')
    
    ax.set_xlabel('Model', fontweight='bold')
    ax.set_ylabel('Score', fontweight='bold')
    ax.set_title('Model Comparison - Average Metrics', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df_comparison['Model'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'model_comparison_{timestamp}.png'), dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Set TRAIN_ALL_MODELS = True to compare all models")

NameError: name 'MODELS' is not defined

## 12. Next Steps and Notes

### For Continuous Training (CI/CD Pipeline):

1. **Jenkins/Cronjob Setup**:
   - Schedule this notebook to run periodically (e.g., weekly, monthly)
   - Use `papermill` to execute notebook programmatically:
     ```bash
     papermill ML_Model_Training.ipynb output_notebook.ipynb \
       -p SELECTED_MODEL 'random_forest' \
       -p TEST_SIZE 0.2
     ```

2. **Model Versioning**:
   - Models are saved with timestamps
   - Keep track of best performing models
   - Implement model registry for production deployment

3. **Data Updates**:
   - As new data chunks are added to `output/` folder, re-run this notebook
   - Models will automatically train on updated dataset

4. **Future Enhancements**:
   - Add temperature data when available
   - Implement hyperparameter tuning (GridSearchCV, RandomizedSearchCV)
   - Add cross-validation for more robust evaluation
   - Implement advanced models (XGBoost, LightGBM, Neural Networks)
   - Add feature importance analysis
   - Implement A/B testing for model comparison in production

5. **Model Selection Tips**:
   - Start with `random_forest` for baseline
   - Use `gradient_boosting` for potentially better accuracy
   - Try `linear_regression` or `ridge` for interpretability
   - Consider ensemble methods (combining multiple models)

### Current Limitations:
- No temperature data (add when available)
- Basic feature engineering (can be enhanced)
- No hyperparameter tuning (using default/reasonable values)
- No cross-validation (single train-test split)

### Recommended Next Actions:
1. Run this notebook with different models
2. Compare results using the "Compare All Models" section
3. Select best performing model based on R² and domain knowledge
4. Set up automated retraining schedule
5. Monitor model performance over time